출처 - 메타코드m

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터프레임 탐색

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px

In [6]:
df = pd.read_csv("/content/drive/MyDrive/메타코드/데이터분석/product_mart.csv")
df.head()

,order_id,quantity,price,discount_ratio,cost,category,product_id,order_date
0,1.0,23.0,37,0.131128,32,e,17.0,2023-02-07
1,150.0,23.0,37,0.072477,32,e,17.0,2023-01-18
2,811.0,23.0,37,0.110052,32,e,17.0,2023-02-22
3,1518.0,23.0,37,0.089504,32,e,17.0,2023-02-05
4,1632.0,23.0,37,0.139912,32,e,17.0,2023-01-31


In [7]:
df.describe()

,order_id,quantity,price,discount_ratio,cost,product_id
count,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000
mean,2500.500000,6.068200,96.41680,0.014351,70.400400,10.476800
std,1443.520003,5.405348,110.90184,0.021502,94.493688,5.774646
min,1.000000,1.000000,27.00000,0.000000,11.000000,1.000000
25%,1250.750000,3.000000,44.00000,0.002250,23.000000,6.000000
50%,2500.500000,4.000000,55.00000,0.007138,35.000000,10.000000
75%,3750.250000,8.000000,63.00000,0.017626,40.000000,16.000000
max,5000.000000,46.000000,365.00000,0.302786,296.000000,20.000000


In [10]:
df['order_date'] = pd.to_datetime(df.order_date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        5000 non-null   float64       
 1   quantity        5000 non-null   float64       
 2   price           5000 non-null   int64         
 3   discount_ratio  5000 non-null   float64       
 4   cost            5000 non-null   int64         
 5   category        5000 non-null   object        
 6   product_id      5000 non-null   float64       
 7   order_date      5000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 312.6+ KB


In [11]:
df.groupby(by=["category", "product_id"]).size()

category  product_id
a         2.0           226
          6.0           269
          7.0           249
          9.0           277
          10.0          224
          13.0          248
b         5.0           257
          18.0          241
c         14.0          205
d         3.0           239
          11.0          259
          16.0          256
e         1.0           273
          4.0           247
          8.0           255
          17.0          252
          20.0          256
f         15.0          261
g         12.0          253
          19.0          253
dtype: int64

# 필요한 컬럼 만들기

In [13]:
# 비율 계산은 집계가 완료된 다음에 진행하자
# 비율의 통계치는 특수한 목적을 가지고 진행해야하는 부분이다.

df.columns

Index(['order_id', 'quantity', 'price', 'discount_ratio', 'cost', 'category',
       'product_id', 'order_date'],
      dtype='object')

In [16]:
# 할인된 가격
df['net_price'] = df.price * (1 - df.discount_ratio)

# 총합
# 할인된 가격 총합
df['total_net_revenue'] = df.net_price * df.quantity

# 총이익 총합
df['total_revenue'] = df.price * df.quantity

# 총비용
df['total_cost'] = df.cost * df.quantity

# 할인 가격
df['discount'] = df.discount_ratio * df.price

# 할인가격 총합
df['total_discount'] = df.discount * df.quantity

df.head()

,order_id,quantity,price,discount_ratio,cost,category,product_id,order_date,net_price,total_net_revenue,total_revenue,total_cost,discount,total_discount
0,1.0,23.0,37,0.131128,32,e,17.0,2023-02-07,32.148266,739.410122,851.0,736.0,4.851734,111.589878
1,150.0,23.0,37,0.072477,32,e,17.0,2023-01-18,34.318335,789.321712,851.0,736.0,2.681665,61.678288
2,811.0,23.0,37,0.110052,32,e,17.0,2023-02-22,32.928086,757.345983,851.0,736.0,4.071914,93.654017
3,1518.0,23.0,37,0.089504,32,e,17.0,2023-02-05,33.688363,774.832339,851.0,736.0,3.311637,76.167661
4,1632.0,23.0,37,0.139912,32,e,17.0,2023-01-31,31.823242,731.934568,851.0,736.0,5.176758,119.065432


In [21]:
df_agg = df.groupby(by= [ "product_id", "category" ,'price'])[[
    "quantity"
    , "total_net_revenue"
    , "total_revenue"
    , "total_cost"
    , "total_discount"]].sum().reset_index()
df_agg.head()

,product_id,category,price,quantity,total_net_revenue,total_revenue,total_cost,total_discount
0,1.0,e,38,3423.0,125043.635382,130074.0,78729.0,5030.364618
1,2.0,a,64,925.0,58583.824672,59200.0,32375.0,616.175328
2,3.0,d,37,1666.0,60407.890501,61642.0,38318.0,1234.109499
3,4.0,e,58,1365.0,77894.667334,79170.0,51870.0,1275.332666
4,5.0,b,55,640.0,35014.789431,35200.0,20480.0,185.210569


In [22]:
# 순이익 비율
df_agg["net_profit_ratio"] = 1 - (df_agg.total_cost + df_agg.total_discount) / df_agg.total_revenue

In [23]:
# 순이익 금액 총합
df_agg["total_net_profit"] = df_agg.total_revenue - df_agg.total_cost - df_agg.total_discount

In [24]:
# 할인율
df_agg['discount_ratio'] = df_agg.total_discount / df_agg.total_revenue

In [25]:
# 비용 비율
df_agg['cost_ratio'] = df_agg.total_cost / df_agg.total_revenue

In [27]:
df_agg.net_profit_ratio + df_agg.cost_ratio + df_agg.discount_ratio

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0


In [29]:
df_agg.total_revenue - df_agg.total_cost - df_agg.total_discount - df_agg.total_net_profit

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [26]:
df_agg.head()

,product_id,category,price,quantity,total_net_revenue,total_revenue,total_cost,total_discount,net_profit_ratio,total_net_profit,discount_ratio,cost_ratio
0,1.0,e,38,3423.0,125043.635382,130074.0,78729.0,5030.364618,0.356064,46314.635382,0.038673,0.605263
1,2.0,a,64,925.0,58583.824672,59200.0,32375.0,616.175328,0.442717,26208.824672,0.010408,0.546875
2,3.0,d,37,1666.0,60407.890501,61642.0,38318.0,1234.109499,0.358358,22089.890501,0.020021,0.621622
3,4.0,e,58,1365.0,77894.667334,79170.0,51870.0,1275.332666,0.328719,26024.667334,0.016109,0.655172
4,5.0,b,55,640.0,35014.789431,35200.0,20480.0,185.210569,0.412920,14534.789431,0.005262,0.581818


# 시각화

In [39]:
fig = px.scatter(
    df_agg
    , x = "total_revenue"
    , y = "net_profit_ratio"
    , width = 700
    , height = 500
    , title = "제품 포트폴리오 분석"
    , hover_name = "product_id"
    , size = "total_net_profit"
    , color = "discount_ratio"
)


fig.show()